<a href="https://colab.research.google.com/github/Voldemort-vk/WIDS---2025/blob/main/Pipeline_using_Hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tokenizers torch

In [2]:
import transformers
print(transformers.__version__)

4.57.3


In [15]:
from datasets import load_dataset
dataset = load_dataset("imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [16]:
import re

def Preprocessing(Review_set):
  Review_set["text"] = re.sub(r"<.*?>", "", Review_set["text"])  #REMOVE HTML TAGS
  return Review_set

dataset = dataset.map(Preprocessing)

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
!pip install --upgrade transformers

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  _, _, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["test"], compute_metrics=compute_metrics)
trainer.train()

In [ ]:
resut = model.evaluate()
print(result)

In [ ]:
model.save_pretrained("./fine_tuned_model")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

loaded_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
loaded_model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval()

review = "The movie was absolutely beautiful:)"

tokenized_review = loaded_tokenizer([review], padding="max_length", truncation=True)

input_ids = tokenized_review["input_ids"].to(device)
attention_mask = tokenized_review["attention_mask"].to(device)

with torch.no_grad():
    outputs = loaded_model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    pred = logits.argmax(-1).item()

label_map = {0: "Negative", 1: "Positive"}
print(label_map[pred])
